# MINOTAUR Quickstart Guide

This notebook demonstrates the basic usage of the MINOTAUR turbofan cycle solver Python interface.

In [ ]:
import minotaur
import numpy as np
import matplotlib.pyplot as plt

print(f"MINOTAUR Python interface v{minotaur.__version__}")

## 1. Single-Point Solution

Solve a single turbofan cycle operating point with specified flight conditions and cycle parameters.

In [ ]:
# Flight conditions
mach = 0.65      # Flight Mach number
alt_km = 8.0     # Altitude in kilometers

# Cycle parameters
bpr = 0.6        # Bypass ratio
opr = 8.0        # Overall pressure ratio

# Solve
result = minotaur.solve(mach=mach, alt_km=alt_km, bpr=bpr, opr=opr)

print(f"Status: {result.status} ({'converged' if result.converged else 'failed'})")
print(f"Iterations: {result.iterations}")
print(f"T4 (turbine inlet): {result.t4:.1f} K")
print(f"TSFC proxy: {result.tsfc_proxy:.4f}")
print(f"Thrust proxy: {result.thrust_proxy:.4f}")

## 2. Parameter Sweep

Sweep over BPR and OPR ranges to generate performance maps.

In [ ]:
# Define sweep ranges
bpr_values = np.linspace(0.2, 1.2, 21)
opr_values = np.linspace(4.0, 14.0, 21)

# Run sweep
results = minotaur.sweep(bpr_values, opr_values, mach=0.65, alt_km=8.0)

# Check convergence
n_total = len(results['status'])
n_converged = np.sum(results['status'] == minotaur.STATUS_OK)
print(f"Converged: {n_converged}/{n_total} ({100*n_converged/n_total:.1f}%)")

In [ ]:
# Reshape for contour plotting
n_bpr = results['n_bpr']
n_opr = results['n_opr']

BPR = results['bpr'].reshape(n_opr, n_bpr)
OPR = results['opr'].reshape(n_opr, n_bpr)
TSFC = results['tsfc'].reshape(n_opr, n_bpr)
T4 = results['t4'].reshape(n_opr, n_bpr)
STATUS = results['status'].reshape(n_opr, n_bpr)

# Mask non-converged points
TSFC_masked = np.ma.masked_where(STATUS != 0, TSFC)
T4_masked = np.ma.masked_where(STATUS != 0, T4)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# TSFC contour
ax1 = axes[0]
cs1 = ax1.contourf(BPR, OPR, TSFC_masked, levels=20, cmap='viridis')
ax1.set_xlabel('Bypass Ratio (BPR)')
ax1.set_ylabel('Overall Pressure Ratio (OPR)')
ax1.set_title('TSFC Proxy')
plt.colorbar(cs1, ax=ax1)

# T4 contour
ax2 = axes[1]
cs2 = ax2.contourf(BPR, OPR, T4_masked, levels=20, cmap='plasma')
ax2.set_xlabel('Bypass Ratio (BPR)')
ax2.set_ylabel('Overall Pressure Ratio (OPR)')
ax2.set_title('Turbine Inlet Temperature T4 [K]')
plt.colorbar(cs2, ax=ax2)

plt.tight_layout()
plt.show()

## 3. Sensitivity Analysis

Compute local sensitivities of outputs with respect to input parameters using finite differences.

In [ ]:
sens = minotaur.sensitivity(mach=0.65, alt_km=8.0, bpr=0.6, opr=8.0)

print("Parameters:", sens['parameters'])
print("Outputs:", sens['outputs'])
print(f"\nBase values: TSFC={sens['base_tsfc']:.4f}, Thrust={sens['base_thrust']:.4f}, T4={sens['base_t4']:.1f} K")
print("\nJacobian (dOutput/dParam):")

jacobian = sens['jacobian']
params = sens['parameters']
outputs = sens['outputs']

# Print as table
print(f"{'Parameter':<12} {'dTSFC':<12} {'dThrust':<12} {'dT4':<12}")
print("-" * 48)
for i, param in enumerate(params):
    print(f"{param:<12} {jacobian[i, 0]:<12.4f} {jacobian[i, 1]:<12.4f} {jacobian[i, 2]:<12.2f}")

In [ ]:
# Visualize sensitivities
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(params))
width = 0.25

# Normalize for visualization
J_norm = jacobian.copy()
for j in range(3):
    max_val = np.max(np.abs(J_norm[:, j]))
    if max_val > 0:
        J_norm[:, j] /= max_val

bars1 = ax.bar(x - width, J_norm[:, 0], width, label='TSFC', color='steelblue')
bars2 = ax.bar(x, J_norm[:, 1], width, label='Thrust', color='darkorange')
bars3 = ax.bar(x + width, J_norm[:, 2], width, label='T4', color='forestgreen')

ax.set_xlabel('Parameter')
ax.set_ylabel('Normalized Sensitivity')
ax.set_title('Parameter Sensitivities (Normalized)')
ax.set_xticks(x)
ax.set_xticklabels(params, rotation=45, ha='right')
ax.legend()
ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

## 4. Degradation Analysis

Compare nominal vs degraded engine performance.

In [ ]:
# Compare degradation levels
levels = ['light', 'moderate', 'severe']
results_deg = []

for level in levels:
    nom, deg, dtsfc, dthrust, dt4 = minotaur.compare_degradation(
        mach=0.65, alt_km=8.0, bpr=0.6, opr=8.0, degradation_level=level
    )
    results_deg.append({
        'level': level,
        'tsfc_change': dtsfc,
        'thrust_change': dthrust,
        't4_change': dt4
    })
    print(f"{level.capitalize():>10}: TSFC {dtsfc:+.2f}%, Thrust {dthrust:+.2f}%, T4 {dt4:+.1f} K")

In [ ]:
# Visualize degradation effects
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

x = np.arange(len(levels))
level_labels = [r['level'].capitalize() for r in results_deg]

# TSFC change
tsfc_changes = [r['tsfc_change'] for r in results_deg]
axes[0].bar(x, tsfc_changes, color='crimson')
axes[0].set_xticks(x)
axes[0].set_xticklabels(level_labels)
axes[0].set_ylabel('TSFC Change [%]')
axes[0].set_title('Fuel Consumption Impact')
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)

# Thrust change
thrust_changes = [r['thrust_change'] for r in results_deg]
axes[1].bar(x, thrust_changes, color='steelblue')
axes[1].set_xticks(x)
axes[1].set_xticklabels(level_labels)
axes[1].set_ylabel('Thrust Change [%]')
axes[1].set_title('Thrust Impact')
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)

# T4 change
t4_changes = [r['t4_change'] for r in results_deg]
axes[2].bar(x, t4_changes, color='darkorange')
axes[2].set_xticks(x)
axes[2].set_xticklabels(level_labels)
axes[2].set_ylabel('T4 Change [K]')
axes[2].set_title('Turbine Temperature Impact')
axes[2].axhline(y=0, color='k', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

## 5. Extended Solver Options

Use `solve_extended` for advanced component models and loss parameters.

In [ ]:
# Standard model
result_std = minotaur.solve_extended(
    mach=0.65, alt_km=8.0, bpr=0.6, opr=8.0,
    compressor_model=0,  # Standard (isentropic)
    turbine_model=0,     # Standard
    nozzle_model=0       # Standard
)

# Advanced model with losses
result_adv = minotaur.solve_extended(
    mach=0.65, alt_km=8.0, bpr=0.6, opr=8.0,
    compressor_model=1,  # Advanced (polytropic)
    turbine_model=1,     # Advanced (cooled)
    nozzle_model=1,      # Advanced (divergence losses)
    inlet_loss=0.02,
    burner_loss=0.04,
    turbine_mech_loss=0.02,
    nozzle_loss=0.01
)

print("Standard Model:")
print(f"  T4={result_std.t4:.1f} K, TSFC={result_std.tsfc_proxy:.4f}, Thrust={result_std.thrust_proxy:.4f}")
print("\nAdvanced Model with Losses:")
print(f"  T4={result_adv.t4:.1f} K, TSFC={result_adv.tsfc_proxy:.4f}, Thrust={result_adv.thrust_proxy:.4f}")

## 6. Status Codes

The solver returns structured status codes to indicate success or failure modes.

In [ ]:
status_codes = {
    minotaur.STATUS_OK: "Converged successfully",
    minotaur.STATUS_MAXITER: "Maximum iterations reached",
    minotaur.STATUS_DIVERGED: "Solution diverged",
    minotaur.STATUS_INVARIANT_VIOL: "Conservation law violation",
    minotaur.STATUS_CONSTRAINT_VIOL: "Constraint violation (T4 > T4_max)",
    minotaur.STATUS_NONPHYSICAL: "Non-physical state (NaN/Inf)"
}

print("MINOTAUR Status Codes:")
print("-" * 50)
for code, desc in status_codes.items():
    print(f"  {code}: {desc}")